In [10]:
import sys 
sys.path.insert(0, "../../")

from src.neo4j_functions import Neo4jConnection
from src import ReaderMetrics

import json
import requests
from langchain_huggingface import HuggingFaceEmbeddings
import os
import transformers
import torch
import requests
from time import sleep
from typing import List, Dict, Tuple
import hashlib
from time import sleep, time
from tqdm import tqdm
from collections import Counter
import numpy as np
import gc
import torch
import joblib
import chromadb
from scipy.spatial import distance

####

EVAL_DATADIR = '../../data/qa_eval'

####

LLM_URL = "https://3090-83-69-192-100.ngrok-free.app/llama" 

NEO4J_URL ="bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PWD = "neo4j"
NEO4J_DBNAME = "neo4j"

####

LOG_DIR = "./logs"
SAVE_STAGE1_LOGDIR = f"{LOG_DIR}/stage1"
SAVE_STAGE2_LOGDIR = f"{LOG_DIR}/stage2"
SAVE_STAGE3_LOGDIR = f"{LOG_DIR}/stage3"
SAVE_STAGE3_1_LOGDIR = f"{LOG_DIR}/stage3_1"
SAVE_STAGE4_LOGDIR = f"{LOG_DIR}/stage4"
SAVE_SCORES_LOGDIR = f"{LOG_DIR}/final"

LOG_TMPDATA_DIRNAME = "tmp"
LOG_STATSDATA_DIRNAME = "stats"
LOG_META_FILENMAE = "metadata.json"
LOG_SCORES_DIRNAME = 'scores'

In [2]:
gc.collect()

25

##### Utils

In [3]:
def generate(prompt: str) -> str:
    flag = True
    while flag:    
        try:
            response = requests.post(LLM_URL, params = {"prompt": prompt})
            resp = response.json()["response"]
            flag = False
        except (requests.ConnectionError, requests.ReadTimeout) as e:
            print("Connection error: ", str(e))
            sleep(1)

    return resp

def check_create_dir(new_dir_path: str):
    if os.path.exists(new_dir_path):
        print("Директория существует")
        raise ValueError
    else:
        os.mkdir(new_dir_path)

def save_json(data: Dict[str, object], save_path: str):
    dump = json.dumps(data, ensure_ascii=False, indent=1)
    with open(save_path, 'w', encoding='utf-8') as fd:
        fd.write(dump)

def load_json(load_path: str) -> Dict[str,object]:
    with open(load_path, 'r', encoding='utf-8') as fd:
        data = json.loads(fd.read())
    return data

def round5(number: float) -> float:
    return round(number, 5)

#### Stage 1: extracting entities from questions

In [5]:
PROMPT_ENTITIES_EXTRACTION_TEMPLATE = '''You are an expert system that can extract key entities from text. Key entities is a noun or an object like persone, device, company and etc. Extract such entities from the given text and present the results in the following format: <entitie1> | <entitie2> | ... | <entitieN>. Generate only entities and dont return some additional text. Examples of texts and extracted entities are listed below:
Text 1: Kayla has positive, negative or neutral opinion about video of Xiaomi 10Pro?
Entities 1: Kayla | opinion | video | Xiaomi 10Pro.
Text 2: Which device is better in battery life: Apple or k30u?
Entities 2: device | battery life | Apple | k30u.
Text 3: The majority of speakers have positive, neutral or negative sentiment about screen of Samsung?
Entities 3: speakers | sentiment | screen | Samsung.
Text 4: Which people have positive opinion about video of Xiaomi 10Pro on 25.11.2020?
Entities 4: people | opinion | video | Xiaomi 10Pro | 25.11.2020.

Text: {text}
Entities: '''

S1_META = {
    'SAVE_ENTITIES_VERSION': "v2",
    'EXTRACT_PROMPT_TEMPLATE': PROMPT_ENTITIES_EXTRACTION_TEMPLATE,
    'ENTITIES_PER_QUESTION': {}
}

In [6]:
def postproc_entity_extraction(raw_text: str) -> List[str]:    
    return list(filter(lambda item: len(item) > 0, list(map(lambda item: item.strip(), raw_text.split('|')))))

def extract_entities_from_question_file(qa_file: str, save_log_statsdata_dir: str, save_log_tmpdata_dir: str):
    print(qa_file)
    data = load_json(f"{EVAL_DATADIR}/{qa_file}")

    start_time = time()
    question_entities = []
    for qa_pair in tqdm(data):
        prompt = S1_META['EXTRACT_PROMPT_TEMPLATE'].format(text=qa_pair['question'])
        entities = postproc_entity_extraction(generate(prompt))
        #print(entities)
        question_entities.append({'question_entities': entities})
    end_time = time()

    save_json(question_entities, f"{save_log_tmpdata_dir}/{qa_file}")

    amout_entities_per_q = list(map(lambda v: len(v['question_entities']), question_entities))
    stats = {
        'mean': np.mean(amout_entities_per_q), 'median': np.median(amout_entities_per_q), 'std': np.std(amout_entities_per_q),
        'min': min(amout_entities_per_q), 'max': max(amout_entities_per_q),
        'elapsed_time (sec)': int(end_time - start_time)}
    
    save_json(stats, f"{save_log_statsdata_dir}/{qa_file}")

In [7]:
save_log_dir = f"{SAVE_STAGE1_LOGDIR}/{S1_META['SAVE_ENTITIES_VERSION']}" 
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_TMPDATA_DIRNAME}"
save_log_statsdata_dir = f"{save_log_dir}/{LOG_STATSDATA_DIRNAME}"
save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"

In [7]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)
check_create_dir(save_log_statsdata_dir)

In [8]:
qa_files = os.listdir(EVAL_DATADIR)

In [ ]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in qa_files[6:]:
    extract_entities_from_question_file(qa_file, save_log_statsdata_dir, 
                                        save_log_tmpdata_dir)
    gc.collect()

save_json(S1_META, save_log_metafile)

#### Stage 2: connecting entities with nodes

In [5]:
S2_META = {
    "LOAD_ENTITIES_VERSION": "v2",
    "LOAD_NODES_DB_VERSION": "v7",
    "SAVE_CONN_NODES_VERSION": "v8",
    "THRESHOLD": 0.5,
    "MAX_K": 1,
    'db_location': 'local'
}

NODES_DB_PATH = f'../../data/vectorized_nodes/{S2_META["LOAD_NODES_DB_VERSION"]}/densedb'
NODES_DB_META = load_json(f'../../data/vectorized_nodes/{S2_META["LOAD_NODES_DB_VERSION"]}/operation_info.json')

save_log_dir = f"{SAVE_STAGE2_LOGDIR}/{S2_META['SAVE_CONN_NODES_VERSION']}" 
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_TMPDATA_DIRNAME}"
save_log_statsdata_dir = f"{save_log_dir}/{LOG_STATSDATA_DIRNAME}"
save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"
load_log_tmpdata_dir = f"{SAVE_STAGE1_LOGDIR}/{S2_META['LOAD_ENTITIES_VERSION']}/{LOG_TMPDATA_DIRNAME}"


In [6]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)
check_create_dir(save_log_statsdata_dir)

In [6]:
client = chromadb.PersistentClient(path=NODES_DB_PATH)
NODES_COLLECTION = client.get_collection(name=NODES_DB_META["data_name"])

ENCODE_KWARGS = NODES_DB_META["encode_kwargs"]
ENCODE_KWARGS['prompt'] = NODES_DB_META["encode_prompts"]['query']

EMBEDDER = HuggingFaceEmbeddings(
    model_name=NODES_DB_META["model_name"],
    model_kwargs={'device': 'cuda'},
    encode_kwargs=ENCODE_KWARGS
    )

No sentence-transformers model found with name /home/dzigen/Desktop/PersonalAI/Personal-AI/models/intfloat/multilingual-e5-small. Creating a new one with MEAN pooling.


In [7]:
def connect_nodes_with_entities(qa_file: str, load_log_tmpdata_dir: str, save_log_statsdata_dir: str, save_log_tmpdata_dir: str):
    print(qa_file)
    entities_data = load_json(f"{load_log_tmpdata_dir}/{qa_file}")
    
    start_time = time()
    connected_nodes = []
    for entity_item in tqdm(entities_data):
        tmp_nodes = []
        for entity in entity_item['question_entities']:
            query_embedding = EMBEDDER.embed_query(entity)

            docs_with_scores = NODES_COLLECTION.query(
                query_embeddings=[query_embedding],
                include=["documents", "metadatas", "distances"],
                n_results=50)
            
            filtered_docs_id = list(filter(lambda i: docs_with_scores['distances'][0][i] < S2_META['THRESHOLD'], 
                                           range(len(docs_with_scores['documents'][0]))))
            
            retrieved_nodes = list(map(lambda i: (
                docs_with_scores['distances'][0][i], docs_with_scores['documents'][0][i], 
                docs_with_scores['metadatas'][0][i]['node_id']), filtered_docs_id)) 
            
            if S2_META['MAX_K'] > 0:
                retrieved_nodes = retrieved_nodes[:S2_META['MAX_K']]

            tmp_nodes += retrieved_nodes
        
        unique_nodes = list(set(tmp_nodes))
        connected_nodes.append(unique_nodes)
    end_time = time()

    save_json(connected_nodes, f"{save_log_tmpdata_dir}/{qa_file}")

    amout_nodes_per_q = list(map(lambda v: len(v), connected_nodes))
    stats = {
        'mean': np.mean(amout_nodes_per_q), 'median': np.median(amout_nodes_per_q), 'std': np.std(amout_nodes_per_q),
        'min': min(amout_nodes_per_q), 'max': max(amout_nodes_per_q),
        'elapsed_time': int(end_time - start_time)}

    save_json(stats, f"{save_log_statsdata_dir}/{qa_file}")
    

In [ ]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in qa_files:
    connect_nodes_with_entities(qa_file, load_log_tmpdata_dir, 
                                save_log_statsdata_dir, save_log_tmpdata_dir)
    gc.collect()

save_json(S2_META, save_log_metafile)

#### Stage 3: extracting relevant triplets from knowledge graph

In [8]:
S3_META = {
    "LOAD_CONN_NODES_VERSION": "v8",
    "LOAD_NODES_DB_VERSION": "v7",
    'LOAD_SHORT_DISTS_VERSION': 'v1',
    "SAVE_TRIPLETS_VERSION": "v12",
    "MAX_DEPTH": 25,
    "MAX_WIDTH": -1,
    "SAMPLE_MAX_SIZE": 50,
    'D_METRIC': 'constant',
    'H_METRIC': 'weight_with_short_path',
    'STRINGIFY_TRIPLETS_VERSION': 'v1'
}

def stringify_v1(triplete):
    node1_str = f"{list(triplete['a'].labels)[0]}: {triplete['a']['name']}"
    node2_str = f"{list(triplete['b'].labels)[0]}: {triplete['b']['name']}"
    if node2_str < node1_str:
        node1_str, node2_str = node2_str, node1_str
        
    relation_str = triplete['rel'].type
    if len(triplete['rel'].keys()) > 0:
        relation_str = ', '.join([f"{k}: {v}" for k, v in triplete['rel'].items()])
    return ' | '.join([node1_str, relation_str, node2_str])

STRINGIFY_FUNC_MAP = {
    'v1':  stringify_v1
}

#NODES_DB_PATH = f'../../data/vectorized_nodes/{S3_META["LOAD_NODES_DB_VERSION"]}/densedb'
NODES_DB_META = load_json(f'../../data/vectorized_nodes/{S3_META["LOAD_NODES_DB_VERSION"]}/operation_info.json')
NODES_V_DIST_INFO = joblib.load(f'../../data/vectorized_nodes/{S3_META["LOAD_NODES_DB_VERSION"]}/nodes_distances_matrix')
NODES_SHORT_DIST_INFO = joblib.load(f"../../data/graph_short_paths/stage2/{S3_META['LOAD_SHORT_DISTS_VERSION']}/distances_matrix")

def get_nodes_path(parent, U, end_node, spare_closest_node):
    if end_node['id'] not in parent.keys():
        node = spare_closest_node
    else:
        node = end_node  
    
    path, end_flag, cur_n = [node], False, node['id']
    while not end_flag:
        next_n = parent[cur_n]
        if next_n is None:
            end_flag = True
        else:
            path.append(U[next_n])
            cur_n = next_n

    return path

def precomputed_dist(node1, node2, U, parent):
    return NODES_V_DIST_INFO['MATRIX'][NODES_V_DIST_INFO['ID_TO_INDEX_MAP'][node1['id']]][NODES_V_DIST_INFO['ID_TO_INDEX_MAP'][node2['id']]]
    
def weighted_short_path(node1, node2, U, parent):
    short_dist = NODES_SHORT_DIST_INFO['MATRIX'][NODES_SHORT_DIST_INFO['ID_TO_INDEX_MAP'][node1['id']]][NODES_SHORT_DIST_INFO['ID_TO_INDEX_MAP'][node2['id']]]
    w = precomputed_dist(node1, node2, U, parent)
    return short_dist * w

def avg_weighted_short_path(node1, node2, U, parent):
    nodes_path = get_nodes_path(parent, U, node1, None)
    acc_dist = 0
    for i in range(len(nodes_path)-1):
        acc_dist += precomputed_dist(nodes_path[i], nodes_path[i+1], U, parent)
    acc_dist += precomputed_dist(node1, node2, U, parent)

    short_dist = NODES_SHORT_DIST_INFO['MATRIX'][NODES_SHORT_DIST_INFO['ID_TO_INDEX_MAP'][node1['id']]][NODES_SHORT_DIST_INFO['ID_TO_INDEX_MAP'][node2['id']]]
    return np.mean(acc_dist) * short_dist 

distance_METRICS = {
    'l2': precomputed_dist,
    'ip': precomputed_dist,
    'constant': lambda v1, v2, U, parent: 1,
    'weight_with_short_path': weighted_short_path,
    'avg_weighted_with_short_path': avg_weighted_short_path,
}

D_METRIC = distance_METRICS[S3_META['D_METRIC']]
H_METRIC = distance_METRICS[S3_META['H_METRIC']]

In [9]:
#client = chromadb.PersistentClient(path=NODES_DB_PATH)
#NODES_COLLECTION = client.get_collection(name=NODES_DB_META["data_name"])

GRAPH_MODEL = Neo4jConnection(uri=NEO4J_URL, user=NEO4J_USER, pwd=NEO4J_PWD)

In [10]:
def get_nodes_tripletes(nodes_path: List[Dict], graph_model) -> List[Tuple[Dict, Dict, Dict]]:
    tripletes = {}

    if len(nodes_path) > 1:
        for i in range(len(nodes_path)-1):
            node1, node2 = nodes_path[i], nodes_path[i+1]
            relations = graph_model.execute_query(
                f'MATCH (a)-[rel]-(b) WHERE elementId(a) = "{node1["id"]}" AND elementId(b) = "{node2["id"]}"  RETURN a, rel, b',
                db=NEO4J_DBNAME)
                
            for relation in relations:
                formated_node1 = {'name': relation['a']['name'], 'labels': list(relation['a'].labels), 'element_id': relation['a'].element_id}
                formated_node2 = {'name': relation['b']['name'], 'labels': list(relation['b'].labels), 'element_id': relation['b'].element_id}
                formated_relation = {'type': relation['rel'].type, 'element_id': relation['rel'].element_id}
                formated_relation.update(relation['rel'])
                
                triplet_str = STRINGIFY_FUNC_MAP[S3_META['STRINGIFY_TRIPLETS_VERSION']](relation)
                triplet_id = hashlib.md5(triplet_str.encode('utf-8')).hexdigest()

                tripletes[triplet_id] = (formated_node1, formated_relation, formated_node2)
                
    elif len(nodes_path) == 1:
        node1 = nodes_path[0]
        relations = graph_model.execute_query(
            f'MATCH (a)-[rel]-(b) WHERE elementId(a) = "{node1["id"]}" RETURN a, rel, b',
            db=NEO4J_DBNAME)

        for relation in relations:
            formated_node1 = {'name': relation['a']['name'], 'labels': list(relation['a'].labels), 'element_id': relation['a'].element_id}
            formated_node2 = {'name': relation['b']['name'], 'labels': list(relation['b'].labels), 'element_id': relation['b'].element_id}
            formated_relation = {'type': relation['rel'].type, 'element_id': relation['rel'].element_id}
            formated_relation.update(relation['rel'])

            triplet_str = STRINGIFY_FUNC_MAP[S3_META['STRINGIFY_TRIPLETS_VERSION']](relation)
            triplet_id = hashlib.md5(triplet_str.encode('utf-8')).hexdigest()

            tripletes[triplet_id] = (formated_node1, formated_relation, formated_node2)

    return tripletes

In [11]:
def get_min_f_node(Q, f):
    node_ids = list(Q.keys())
    min_node_id = node_ids[0]
    min_f = f[min_node_id]

    for idx in range(1, len(node_ids)):
        if f[node_ids[idx]] < min_f:
            min_node_id = node_ids[idx]
            min_f = f[min_node_id]
            
    return Q[min_node_id]

#def add_v_to_queue(new_v: Dict, Q_queue: List[Dict], f: Dict):
#    flag = False
#    for i in range(len(Q_queue)-1, -1, -1):
#        if f[new_v['id']] <= f[Q_queue[i]['id']]:
#            Q_queue.insert(i, new_v)
#            flag = True
#            break
#   
#    if not flag:
#        Q_queue.insert(0, new_v)

def get_adjecent_nodes(base_node, graph_model, parent):
    #print(node)
    flag = False
    while not flag:
        try:
            raw_nodes = graph_model.execute_query(f'MATCH (a)-[r]-(b) WHERE elementId(a) = "{base_node["id"]}" RETURN b', db=NEO4J_DBNAME)
            formated_nodes = list({node['b'].element_id: {'id': node['b'].element_id, 'name': node['b']['name']} for node in raw_nodes 
                                   if node['b'].element_id != parent[base_node['id']]}.values())
            flag = True
        except TypeError as e:
            print(base_node)
            graph_model.close()
            graph_model = Neo4jConnection(uri=NEO4J_URL, user=NEO4J_USER, pwd=NEO4J_PWD)
    
    #print(base_node, len(raw_nodes), len(formated_nodes ))

    return formated_nodes

def filter_adjenced_nodes(base_node, adj_nodes, distance_metric, U, parent, max_width: int = 10):
    sorted_adj_n_distances = sorted(list(map(lambda node_item: (node_item[0], distance_metric(base_node, node_item[1], U, parent)), enumerate(adj_nodes))), 
                                    key=lambda item: item[1])
    filtered_nodes = [adj_nodes[item[0]] for item in sorted_adj_n_distances[:max_width]]

    return filtered_nodes

def A_star_search(start_node: Dict, end_node: Dict, distance_metric, H_METRIC, graph_model, max_depth: int=5, process=None):
    U, Q, D = {}, {start_node['id']: start_node}, {start_node['id']: 0}
    g = {start_node['id']: 0}
    parent = {start_node['id']: None}
    f = {start_node['id']: g[start_node['id']] + H_METRIC(start_node, end_node, U, parent)}
    
    spare_closest_node = start_node
    while len(Q) != 0:
        if process is not None:
            process.set_postfix({'Q': len(Q)}, refresh=True)
        #print(len(Q))

        current = get_min_f_node(Q, f) # вершина из Q с минимальным значением f
        #print(current, f[current['id']])

        # Сохраняем промежуточную вершину, до которой есть путь. 
        # Если не будет найден путь до end_node, то будет использован путь до spare_closest_node
        if H_METRIC(current, end_node, U, parent) <= H_METRIC(spare_closest_node, end_node, U, parent):
            if f[current['id']] <= f[spare_closest_node['id']]:
                spare_closest_node = current
        
        #
        if current['id'] == end_node['id']:
            break

        del Q[current['id']]
        if D[current['id']] >= max_depth:
            continue
        U[current['id']] = current

        adj_nodes = get_adjecent_nodes(current, graph_model, parent)

        if S3_META['MAX_WIDTH'] > 0:
            adj_nodes = filter_adjenced_nodes(
                current, adj_nodes, distance_metric, S3_META['MAX_WIDTH'], U, parent)

        break_flag = False
        for v in adj_nodes:
            #print(current, v, end_node)
            tentativeScore = g[current['id']] + distance_metric(current, v, U, parent)      
            if (v['id'] in U.keys()) and (tentativeScore >= g[v['id']]):
                continue
            if (v['id'] not in U.keys()) or (tentativeScore < g[v['id']]):
                parent[v['id']] = current['id']
                g[v['id']] = tentativeScore
                f[v['id']] = g[v['id']] + H_METRIC(v, end_node, U, parent)
                D[v['id']] = D[current['id']] + 1

                if v['id'] not in Q.keys():
                    #print(v)
                    Q[v['id']] = v

                #if D[v['id']] < max_depth and v['id'] == end_node['id']:
                #    U[v['id']] = v
                #    del Q[v['id']]
                #    break_flag = True
                #    #print(D[v['id']])
                #    break

        if break_flag:
            break

    return U, Q, D, parent, spare_closest_node

In [12]:
def extract_relevant_tripletes_from_graph(qa_file: str, load_log_tmpdata_dir: str, 
                                          save_log_statsdata_dir: str, save_log_tmpdata_dir: str):
    print(qa_file)

    nodes_data = load_json(f"{load_log_tmpdata_dir}/{qa_file}")

    start_time = time()
    retrieved_tripletes = []
    process = tqdm(nodes_data[:S3_META["SAMPLE_MAX_SIZE"]]) if S3_META["SAMPLE_MAX_SIZE"] > 0 else tqdm(nodes_data)

    for item in process:
        formated_nodes = [{'id': node[2], 'name': node[1]} for node in item]

        tripletes_pool = {}
        if len(formated_nodes) > 1:
            for i in range(len(formated_nodes)-1):
                start_node = formated_nodes[i]
                for j in range(i+1, len(formated_nodes)):
                    end_node = formated_nodes[j]

                    #print("here1")
                    U, Q, D, parent, spare_closest_node = A_star_search(start_node, end_node, D_METRIC, H_METRIC, GRAPH_MODEL, 
                                                    max_depth=S3_META['MAX_DEPTH'])
                    #print("here2")
                    nodes_path = get_nodes_path(parent, U, end_node, spare_closest_node)
                    #print("here3")
                    new_tripletes = get_nodes_tripletes(nodes_path, GRAPH_MODEL) # сразу формируется уникальный (по содержанию) набор триплетов

                    process.set_postfix({"U_amount": len(U), "Q_amount": len(Q), "start_node": start_node['name'], "end_node": end_node['name']})
                    tripletes_pool.update(new_tripletes)

        elif len(formated_nodes) == 1:
            new_tripletes = get_nodes_tripletes(formated_nodes, GRAPH_MODEL)
            tripletes_pool.update(new_tripletes)
                
        retrieved_tripletes.append({'retrieved_tripletes': list(tripletes_pool.values())})
    end_time = time()

    save_json(retrieved_tripletes, f"{save_log_tmpdata_dir}/{qa_file}")

    amout_tripletes_per_q = list(map(lambda v: len(v['retrieved_tripletes']), retrieved_tripletes))
    duplicate_tripletes_per_q = np.mean([np.mean(list(Counter(
        list(map(lambda v2: v2[1]['element_id'], v['retrieved_tripletes']))).values())) for v in retrieved_tripletes])
    stats =  {
        'mean': np.mean(amout_tripletes_per_q), 'median': np.median(amout_tripletes_per_q), 
        'std': np.std(amout_tripletes_per_q), 'min': min(amout_tripletes_per_q),
        'max': max(amout_tripletes_per_q),
        'elapsed_time (sec)': int(end_time - start_time),
        'duplicate_triplets_per_q': duplicate_tripletes_per_q}
    save_json(stats, f"{save_log_statsdata_dir}/{qa_file}")

In [13]:
save_log_dir = f"{SAVE_STAGE3_LOGDIR}/{S3_META['SAVE_TRIPLETS_VERSION']}" 
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_TMPDATA_DIRNAME}"
save_log_statsdata_dir = f"{save_log_dir}/{LOG_STATSDATA_DIRNAME}"
save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"
load_log_tmpdata_dir = f"{SAVE_STAGE2_LOGDIR}/{S3_META['LOAD_CONN_NODES_VERSION']}/{LOG_TMPDATA_DIRNAME}"

In [14]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)
check_create_dir(save_log_statsdata_dir)

In [15]:
gc.collect()

0

In [ ]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in qa_files:
    extract_relevant_tripletes_from_graph(qa_file, load_log_tmpdata_dir, save_log_statsdata_dir, save_log_tmpdata_dir)
    gc.collect()

save_json(S3_META, save_log_metafile)

#### Stage 3.1: Filtering triplets by relevance score

In [204]:
S3_1_META = {
    "LOAD_TRIPLETS_VERSION": "v11",
    "LOAD_TRIPLETS_DB_VERSION": "v1",
    "SAVE_FILTERED_TRIPLETS_VERSION": "v4",
    "TOP_K": 99
}

NODES_DB_PATH = f'../../data/vectorized_triplets/{S3_1_META["LOAD_TRIPLETS_DB_VERSION"]}/densedb'
NODES_DB_META = load_json(f'../../data/vectorized_triplets/{S3_1_META["LOAD_TRIPLETS_DB_VERSION"]}/operation_info.json')

def stringify_v1(triplete):
    node1_str = f"{list(triplete['a'].labels)[0]}: {triplete['a']['name']}"
    node2_str = f"{list(triplete['b'].labels)[0]}: {triplete['b']['name']}"
    if node2_str < node1_str:
        node1_str, node2_str = node2_str, node1_str
        
    relation_str = triplete['rel'].type
    if len(triplete['rel'].keys()) > 0:
        relation_str = ', '.join([f"{k}: {v}" for k, v in triplete['rel'].items()])
    return ' | '.join([node1_str, relation_str, node2_str])

STRINGIFY_FUNC_MAP = {
    'v1':  stringify_v1
}

In [205]:
client = chromadb.PersistentClient(path=NODES_DB_PATH)
NODES_COLLECTION = client.get_collection(name=NODES_DB_META["data_name"])

ENCODE_KWARGS = NODES_DB_META["encode_kwargs"]
ENCODE_KWARGS['prompt'] = NODES_DB_META["encode_prompts"]['query']

EMBEDDER = HuggingFaceEmbeddings(
    model_name=NODES_DB_META["model_name"],
    model_kwargs={'device': 'cuda'},
    encode_kwargs=ENCODE_KWARGS
    )

No sentence-transformers model found with name /home/dzigen/Desktop/PersonalAI/Personal-AI/models/intfloat/multilingual-e5-small. Creating a new one with MEAN pooling.


In [206]:
save_log_dir = f"{SAVE_STAGE3_1_LOGDIR}/{S3_1_META['SAVE_FILTERED_TRIPLETS_VERSION']}" 
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_TMPDATA_DIRNAME}"
save_log_statsdata_dir = f"{save_log_dir}/{LOG_STATSDATA_DIRNAME}"
save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"

load_log_tmpdata_dir = f"{SAVE_STAGE3_LOGDIR}/{S3_1_META['LOAD_TRIPLETS_VERSION']}/{LOG_TMPDATA_DIRNAME}"

In [207]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)
check_create_dir(save_log_statsdata_dir)

In [208]:
def filter_triplets(qa_file, load_log_tmpdata_dir, save_log_tmpdata_dir, save_log_statsdata_dir):
    print(qa_file)

    triplets_data = load_json(f"{load_log_tmpdata_dir}/{qa_file}")
    questions_data = load_json(f"{EVAL_DATADIR}/{qa_file}")
    
    start_time = time()
    filtered_triplets = []
    for question_item, triplets_item in tqdm(zip(questions_data, triplets_data)):
        question_embd = EMBEDDER.embed_query(question_item['question'])
        base_triplets_ids = list(map(lambda v: v[1]['element_id'], triplets_item['retrieved_tripletes']))
        tmp_triplets = []

        if len(base_triplets_ids) > 0:
            relevant_triplets = NODES_COLLECTION.query(
                query_embeddings=[question_embd], n_results=S3_1_META['TOP_K'],
                where={"triplet_id": {"$in": base_triplets_ids}})
            
            for idx, triplet_id in enumerate(base_triplets_ids):
                if triplet_id in relevant_triplets['ids'][0]:
                    tmp_triplets.append(triplets_item['retrieved_tripletes'][idx])
        
        filtered_triplets.append({"filtered_triplets": tmp_triplets})
    end_time = time()

    save_json(filtered_triplets, f"{save_log_tmpdata_dir}/{qa_file}")

    amout_tripletes_per_q = list(map(lambda v: len(v['filtered_triplets']), filtered_triplets))
    duplicate_tripletes_per_q = np.mean([np.mean(list(Counter(
        list(map(lambda v2: v2[1]['element_id'], v['filtered_triplets']))).values())) for v in filtered_triplets])
    stats =  {
        'mean': np.mean(amout_tripletes_per_q), 'median': np.median(amout_tripletes_per_q), 
        'std': np.std(amout_tripletes_per_q), 'min': min(amout_tripletes_per_q),
        'max': max(amout_tripletes_per_q),
        'elapsed_time (sec)': int(end_time - start_time),
        'duplicate_tripletes_per_q': duplicate_tripletes_per_q}
    save_json(stats, f"{save_log_statsdata_dir}/{qa_file}")

In [209]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in qa_files:
    filter_triplets(
        qa_file, load_log_tmpdata_dir, save_log_tmpdata_dir, save_log_statsdata_dir)
    gc.collect()
    
save_json(S3_1_META, save_log_metafile)

which_people_about_device.json


5it [00:00, 45.44it/s]

200it [00:04, 45.81it/s]


compare_questions.json


200it [00:04, 49.15it/s]


...

200it [00:04, 49.21it/s]


dominant_opinion.json


200it [00:04, 49.29it/s]


device_sentiment.json


200it [00:04, 47.22it/s]


#### Stage 4: generating answers for a questions 

In [19]:
PROMPT_QA_TEMPLATE = """Answer the question, based on provided info by analogy with examples given. Generate chain of thought and then give the final answer in the following format:
### Answer
Chain of thought: ... Final answer: ...
Question 1: Whose opinions from Anthony and Grace about devices are most similar to Faith's?
Info 1: person: Anthony, device: Xiaomi, opinion: hard, feature: maintenance point
person: Anthony, device: mate30pro, opinion: not as good as, feature: signal
person: Anthony, device: iPhone, opinion: not as good as, feature: signal
person: Grace, device: Xiaomi 12, opinion: beat, feature: charging speed
person: Faith, device: Xiaomi, opinion: problem, feature: product control
person: Faith, device: k30s, opinion: not as good, feature: film effect
person: Faith, device: red rice, opinion: not as good, feature: film effect
### Answer 1
Chain of thought 1: The task is to compare Anthony's and Grace's opinions to Faith's opinions about devices. Faith has two types of opinions: "problem" with the Xiaomi device and "not as good" with both k30s and red rice devices. We will look for similar expressions of dissatisfaction from Anthony and Grace. Grace's opinion about Xiaomi 12 is "beat," which is not similar to any of Faith's negative opinions. Anthony's opinions include "hard" for Xiaomi and "not as good as" for mate30pro and iPhone with respect to the signal feature. "Not as good as" matches Faith's "not as good."
Final answer 1: Anthony
Question 2: The majority of speakers have positive, neutral or negative sentiment about signal of Apple?
Info 2: person: Alejandro, time: 15.11.2020, opinion: beats, device: Apple, feature: battery life
person: Jacqueline, time: 30.12.2020, opinion: Nice pictures taken, device: Apple, feature: taking pictures
person: Diego, time: 30.12.2020, opinion: Doesnt overheat, device: Apple, feature: heat radiation
person: Lily, time: 30.12.2020, opinion: Not bad, device: Apple, feature: configuration of other processors
person: Margaret, time: 25.11.2020, opinion: Pictures turn blurry, device: Apple, feature: taking pictures
person: Amber, time: 25.11.2020, opinion: Really unhelpful, device: Apple, feature: sales
person: Jessica, time: 25.11.2020, opinion: Always been strong, device: Apple, feature: signal
person: Bernard, time: 25.11.2020, opinion: No lag, device: Apple, feature: play games
### Answer 2
Chain of thought 2: To determine the sentiment about the signal of Apple, we need to find the opinions specifically related to the "signal" feature of Apple. From the provided info, only Jessica's opinion mentions the signal: "Always been strong." This is a positive sentiment. Since there's only one opinion regarding the signal, the majority sentiment is positive.
Final answer 2: Positive
Question 3: {q}
Info 3: {c}
### Answer 3 """

In [20]:
S4_META = {
    "LOAD_FILTERED_TRIPLETS_VERSION": "v2",
    "SAVE_GEN_ANSWERS_VERSION": "v16",
    "QUESTION_PROMPT_TEMPLATE": PROMPT_QA_TEMPLATE,
    "TRPLETE_PROMPT_TEMPLATES": {
        "manufacturer": "device: {d}, company: {c}",
        "opinion": "person: {p}, time: {t}, opinion: {o}, device: {d}, feature: {f}",
        'has_device': "person: {p}, has device: {d}"
    },
    "MAX_LIST_LEN": -1
}

In [21]:
def preproc_tripletes(raw_triplets: List[List[str]]) -> str:
    formated_triplets = []

    if len(raw_triplets) == 0:
        return "empty."

    for triplete in raw_triplets:
        #print(len(triplete))

        if triplete[1]["type"] == "manufacturer":
            device_node, company_node = (triplete[0], triplete[2]) if 'device' in triplete[0]["labels"] else (triplete[2], triplete[0])
            formated_triplete = S4_META["TRPLETE_PROMPT_TEMPLATES"]["manufacturer"].format(d=device_node['name'], c=company_node['name'])

        elif triplete[1]["type"] == "opinion":
            device_node, feature_node = (triplete[0], triplete[2]) if 'device' in triplete[0]["labels"] else (triplete[2], triplete[0])
            relation = triplete[1]
            formated_triplete = S4_META["TRPLETE_PROMPT_TEMPLATES"]["opinion"].format(
                p=relation["person"], o=relation["opinion"].replace('_', ' '),
                t=relation["time"], f=feature_node['name'],d=device_node['name'])
        
        elif triplete[1]["type"] == "has_device":
            device_node, person_node = (triplete[0], triplete[2]) if 'device' in triplete[0]["labels"] else (triplete[2], triplete[0])
            formated_triplete = S4_META["TRPLETE_PROMPT_TEMPLATES"]['has_device'].format(d=device_node['name'], p=person_node['name'])

        else:
            raise ValueError
        
        formated_triplets.append(formated_triplete)

    str_triplets = "\n".join(formated_triplets[:S4_META["MAX_LIST_LEN"]] if S4_META["MAX_LIST_LEN"] > 0 else formated_triplets)
    return str_triplets

def generate_answers_from_triplets_file(qa_file: str, load_log_tmpdata_dir: str, save_log_tmpdata_dir: str):
    print(qa_file)
    triplets_data = load_json(f"{load_log_tmpdata_dir}/{qa_file}")
    questions_data = load_json(f"{EVAL_DATADIR}/{qa_file}")

    generated_answers = []
    for triplete_item, quiestion_item in tqdm(zip(triplets_data, questions_data)):
        context = preproc_tripletes(triplete_item['filtered_triplets'])
        question = quiestion_item['question']
        prompt = S4_META["QUESTION_PROMPT_TEMPLATE"].format(q=question, c=context)
        
        #print()
        #print(prompt)
        
        found_line = ""
        raw_answer = generate(prompt).strip()
        for line in raw_answer.split("\n"):
            if "Final answer 3" in line:
                found_line = line
                break
        if found_line:
            answer = found_line.split("Final answer 3: ")[-1]
        else:
            answer = raw_answer.split("\n")[1]

        #print(answer)

        generated_answers.append({'generated_answer': answer, 'used_prompt': prompt})
        
    save_json(generated_answers, f"{save_log_tmpdata_dir}/{qa_file}")

In [22]:
save_log_dir = f"{SAVE_STAGE4_LOGDIR}/{S4_META['SAVE_GEN_ANSWERS_VERSION']}" 
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_TMPDATA_DIRNAME}"
save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"
load_log_tmpdata_dir = f"{SAVE_STAGE3_1_LOGDIR}/{S4_META['LOAD_FILTERED_TRIPLETS_VERSION']}/{LOG_TMPDATA_DIRNAME}"

In [23]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)

In [24]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in qa_files:
    generate_answers_from_triplets_file(
        qa_file, load_log_tmpdata_dir, save_log_tmpdata_dir)
    gc.collect()
    
save_json(S4_META, save_log_metafile)

which_people_about_device.json


50it [07:00,  8.42s/it]


compare_questions.json


50it [06:56,  8.34s/it]


same_manufacturer.json


...

50it [03:43,  4.48s/it]


dominant_opinion.json


50it [04:19,  5.18s/it]


device_sentiment.json


50it [04:36,  5.54s/it]


#### Stage Final: Measuring quality of generated answers

In [25]:
SF_META = {
    'SAVE_SCORES_VERSION': "v16",
    'LOAD_GEN_ANSWERS_VERSION': "v16",
    'BERT_SCORE_MODEL_PATH': "google/electra-base-discriminator"
}

BASE_DIR = '/home/dzigen/Desktop/PersonalAI/Personal-AI'
METRICS = ReaderMetrics(base_dir=BASE_DIR, bs_model_path=SF_META['BERT_SCORE_MODEL_PATH'])

save_log_dir = f"{SAVE_SCORES_LOGDIR}/{SF_META['SAVE_SCORES_VERSION']}" 
save_log_metafile = f"{save_log_dir}/{LOG_META_FILENMAE}"
save_log_tmpdata_dir = f"{save_log_dir}/{LOG_SCORES_DIRNAME}"
gen_answers_dir = f'{SAVE_STAGE4_LOGDIR}/{SF_META["LOAD_GEN_ANSWERS_VERSION"]}/{LOG_TMPDATA_DIRNAME}'

Loading Meteor...
Loading ExactMatch
Loading BertScore


In [26]:
def measure_quality_from_answers_file(qa_file: str, gen_answers_dir: str, save_log_tmpdata_dir: str):
    print(qa_file)
    generated_answers_data = load_json(f"{gen_answers_dir}/{qa_file}")
    target_answers_data = load_json(f"{EVAL_DATADIR}/{qa_file}")
    
    gen_answers = list(map(lambda item: item['generated_answer'], generated_answers_data)) 
    trgt_answers = list(map(lambda item: item['answer'], target_answers_data))[:len(gen_answers)]

    b1_scores = METRICS.bleu1(gen_answers, trgt_answers)
    b2_scores  = METRICS.bleu2(gen_answers, trgt_answers)
    rl_scores = METRICS.rougel(gen_answers, trgt_answers)
    m_scores = METRICS.meteor(gen_answers, trgt_answers)
    em_scores = METRICS.exact_match(gen_answers, trgt_answers)
    bs_scores = METRICS.bertscore(gen_answers, trgt_answers)

    scores = {
        'BLEU1': str(round5(np.mean(b1_scores))),
        'BLEU2': str(round5(np.mean(b2_scores))),
        'METEOR': str(round5(np.mean(m_scores))),
        'RougeL': str(round5(np.mean(rl_scores))),
        'ExactMatch': str(round5(np.mean(em_scores))),
        'BertScore': {k: str(round5(float(v.mean()))) for k, v in bs_scores.items() if k != 'hash'}
    }

    torch.cuda.empty_cache()
    gc.collect()

    save_json(scores, f"{save_log_tmpdata_dir}/{qa_file}")

In [27]:
check_create_dir(save_log_dir)
check_create_dir(save_log_tmpdata_dir)

In [28]:
qa_files = os.listdir(EVAL_DATADIR)
for qa_file in tqdm(qa_files):
    measure_quality_from_answers_file(qa_file, gen_answers_dir, save_log_tmpdata_dir)

  0%|          | 0/12 [00:00<?, ?it/s]

which_people_about_device.json


  8%|▊         | 1/12 [00:19<03:38, 19.86s/it]

compare_questions.json


 17%|█▋        | 2/12 [00:40<03:21, 20.20s/it]

...

 83%|████████▎ | 10/12 [00:55<00:05,  2.77s/it]

dominant_opinion.json


 92%|█████████▏| 11/12 [00:58<00:02,  2.68s/it]

device_sentiment.json


100%|██████████| 12/12 [01:01<00:00,  5.14s/it]


In [29]:
save_json(SF_META, save_log_metafile)

#### View/analyze logs

In [17]:
FINAL_STAGE_VERSION = 'v12'

final_meta_info = load_json(f"{SAVE_SCORES_LOGDIR}/{FINAL_STAGE_VERSION}/{LOG_META_FILENMAE}")
stage4_meta_info = load_json(f"{SAVE_STAGE4_LOGDIR}/{final_meta_info['LOAD_GEN_ANSWERS_VERSION']}/{LOG_META_FILENMAE}")
stage3_1_meta_info = load_json(f"{SAVE_STAGE3_1_LOGDIR}/{stage4_meta_info['LOAD_FILTERED_TRIPLETS_VERSION']}/{LOG_META_FILENMAE}")
stage3_meta_info = load_json(f"{SAVE_STAGE3_LOGDIR}/{stage3_1_meta_info['LOAD_TRIPLETS_VERSION']}/{LOG_META_FILENMAE}")
stage2_meta_info = load_json(f"{SAVE_STAGE2_LOGDIR}/{stage3_meta_info['LOAD_CONN_NODES_VERSION']}/{LOG_META_FILENMAE}")
stage1_meta_info = load_json(f"{SAVE_STAGE1_LOGDIR}/{stage2_meta_info['LOAD_ENTITIES_VERSION']}/{LOG_META_FILENMAE}")

In [ ]:
stage1_meta_info

In [18]:
qa_files = os.listdir(EVAL_DATADIR)

In [ ]:
qa_file = qa_files[12]
print(qa_file)
questions = load_json(f"{EVAL_DATADIR}/{qa_file}")
data = load_json(f"{SAVE_STAGE4_LOGDIR}/{final_meta_info['LOAD_GEN_ANSWERS_VERSION']}/{LOG_TMPDATA_DIRNAME}/{qa_file}")
scores = load_json(f"{SAVE_SCORES_LOGDIR}/{FINAL_STAGE_VERSION}/{LOG_SCORES_DIRNAME}/{qa_file}")
entities = load_json(f"{SAVE_STAGE1_LOGDIR}/{stage1_meta_info['SAVE_ENTITIES_VERSION']}/{LOG_TMPDATA_DIRNAME}/{qa_file}")
nodes = load_json(f"{SAVE_STAGE2_LOGDIR}/{stage2_meta_info['SAVE_CONN_NODES_VERSION']}/{LOG_TMPDATA_DIRNAME}/{qa_file}")
print(scores)

In [ ]:
QUESTION_IDX = 0

print("ENTITIES:\n", entities[QUESTION_IDX]['question_entities'])
print("NODES:\n", list(map(lambda v: v[1], nodes[QUESTION_IDX])))
print("USED_PROMPT:\n", data[QUESTION_IDX]['used_prompt'])
print()
print("GENERATED_ANSWER:\n", data[QUESTION_IDX]['generated_answer'])
print("TARGET_ANSWER:\n", questions[QUESTION_IDX]['answer'])